In [1]:
import numpy as np

In [1]:
batch_size = 2

anchors = [[5.375, 5.03125], [5.40625, 4.6875], [2.96875, 2.53125], [2.59375, 2.78125], [1.9375, 3.25]]
grid_h = 13
grid_w = 13
n_boxes = 5
n_classes = 10
thresh = 0.6

In [28]:
x = np.zeros((batch_size, n_boxes, 1, grid_h, grid_w), dtype=np.float32)
y = np.zeros((batch_size, n_boxes, 1, grid_h, grid_w), dtype=np.float32)
w = np.zeros((batch_size, n_boxes, 1, grid_h, grid_w), dtype=np.float32)
h = np.zeros((batch_size, n_boxes, 1, grid_h, grid_w), dtype=np.float32)
conf = np.zeros((batch_size, n_boxes, 1, grid_h, grid_w), dtype=np.float32)
prob = np.zeros((batch_size, n_boxes, n_classes, grid_h, grid_w), dtype=np.float32)

In [ ]:
tw = np.zeros(w.shape, dtype=np.float32)
th = np.zeros(h.shape, dtype=np.float32)
tx = np.tile(0.5, x.shape).astype(np.float32)
ty = np.tile(0.5, y.shape).astype(np.float32)

tprob = prob.data.copy()

# box_learning_scale: batch_size x n_boxes x 1 x grid_h x grid_w
box_learning_scale = np.tile(0, x.shape).astype(np.float32)

In [29]:
x_shift = np.broadcast_to(np.arange(grid_w, dtype=np.float32), x.shape[1:])
y_shift = np.broadcast_to(np.arange(grid_h, dtype=np.float32).reshape(grid_h, 1), y.shape[1:])
w_anchor = np.broadcast_to(np.reshape(np.array(anchors, dtype=np.float32)[:, 0], (n_boxes, 1, 1, 1)), w.shape[1:])
h_anchor = np.broadcast_to(np.reshape(np.array(anchors, dtype=np.float32)[:, 1], (n_boxes, 1, 1, 1)), h.shape[1:])

In [35]:
x_shift.shape

(5, 1, 13, 13)

In [30]:
best_ious = []

for batch in range(batch_size):
#     n_truth_boxes = len(t[batch])
    # predicted x for center, normalized by image width
    box_x = (x[batch] + x_shift) / grid_w
    # predicted y for center, normalized by image height
    box_y = (y[batch] + y_shift) / grid_h
    # predicted width, normalized by image width
    box_w = np.exp(w[batch]) * w_anchor / grid_w
    # predicted height, normalized by image height
    box_h = np.exp(h[batch]) * h_anchor / grid_h
    
#     ious = []
#     for truth_index in range(n_truth_boxes):
#         truth_box_x = Variable(np.broadcast_to(np.array(t[batch][truth_index]["x"], dtype=np.float32), box_x.shape))
#         truth_box_y = Variable(np.broadcast_to(np.array(t[batch][truth_index]["y"], dtype=np.float32), box_y.shape))
#         truth_box_w = Variable(np.broadcast_to(np.array(t[batch][truth_index]["w"], dtype=np.float32), box_w.shape))
#         truth_box_h = Variable(np.broadcast_to(np.array(t[batch][truth_index]["h"], dtype=np.float32), box_h.shape))
#         truth_box_x.to_gpu(), truth_box_y.to_gpu(), truth_box_w.to_gpu(), truth_box_h.to_gpu()
#         ious.append(multi_box_iou(Box(box_x, box_y, box_w, box_h), Box(truth_box_x, truth_box_y, truth_box_w, truth_box_h)).data.get())  
#     ious = np.array(ious)
    
    # ious should have shape with n_truth_boxes x n_boxes x 1 x grid_h x grid_w
    
#     best_ious.append(np.max(ious, axis=0))

    # best_iou is append with n_boxse x 1 x grid_h x grid_w
    # each element is the iou between one predicted box and the ground truth box with best iou


# the best_ious should have shape of batch_size x n_boxes x 1 x grid_h x grid_w
best_ious = np.array(best_ious)


In [2]:
# 从预测的conf数据中只取iou达到阈值的部分，复制到tconf中
# 将iou超过阈值的conf_learning_scale
tconf[best_ious > thresh] = conf.data.get()[best_ious > thresh]
conf_learning_scale[best_ious > thresh] = 0

NameError: name 'conf' is not defined

In [ ]:
# abs_anchors is the anchor size normalized by image size
abs_anchors = anchors / np.array([grid_w, grid_h])

In [1]:
for batch in range(batch_size):
    for truth_box in t[batch]:
        # 针对t[batch]这个样本（图像）中的没一个真实box，找到和其相匹配的anchor box，记录为truth_n
        
        # truth_w和truth_h为这个真实box的center坐标，是以cell为单位并且取整了的
        # 因此truth_w和truth_h表示了那个位置的cell应来负责预测这个box
        truth_w = int(float(truth_box["x"]) * grid_w)
        truth_h = int(float(truth_box["y"]) * grid_h)
        truth_n = 0
        best_iou = 0.0
        for anchor_index, abs_anchor in enumerate(abs_anchors):
            iou = box_iou(Box(0, 0, float(truth_box["w"]), float(truth_box["h"])), Box(0, 0, abs_anchor[0], abs_anchor[1]))
            if best_iou < iou:
                best_iou = iou
                truth_n = anchor_index
        
        # 和真实box之间iou最大的那个anchor对应的预测的box负责预测这个真实box，
        box_learning_scale[batch, truth_n, :, truth_h, truth_w] = 1.0
        
        # tx/ty是真实box的中心坐标，是由负责预测这个box的cell归一化得到的
        tx[batch, truth_n, :, truth_h, truth_w] = float(truth_box["x"]) * grid_w - truth_w 
        ty[batch, truth_n, :, truth_h, truth_w] = float(truth_box["y"]) * grid_h - truth_h
        
        # tw/th是按照论文中给出的公式，计算的真实box对应的width和height的预测
        # 取log的效果是越大的box绝对误差的影响越小
        tw[batch, truth_n, :, truth_h, truth_w] = np.log(float(truth_box["w"]) / abs_anchors[truth_n][0])
        th[batch, truth_n, :, truth_h, truth_w] = np.log(float(truth_box["h"]) / abs_anchors[truth_n][1])
        
        # 设置真实的tprob，注意tprob是从预测的prob中复制过来的，这里只设置了truth_n、truth_h以及truth_w对应的tprob
        # 对于其余的tprob不进行设置，因此和prob之间应不产生任何距离
        # 也可以考虑设置一个prob_learning_scale，只对有truth box的部分设置为1,其余为0
        tprob[batch, truth_n, :, truth_h, truth_w] = 0
        tprob[batch, truth_n, int(truth_box["label"]), truth_h, truth_w] = 1

        
        full_truth_box = Box(float(truth_box["x"]), float(truth_box["y"]), float(truth_box["w"]), float(truth_box["h"]))
        predicted_box = Box(
            (x[batch][truth_n][0][truth_h][truth_w].data.get() + truth_w) / grid_w, 
            (y[batch][truth_n][0][truth_h][truth_w].data.get() + truth_h) / grid_h,
            np.exp(w[batch][truth_n][0][truth_h][truth_w].data.get()) * abs_anchors[truth_n][0],
            np.exp(h[batch][truth_n][0][truth_h][truth_w].data.get()) * abs_anchors[truth_n][1]
        )
        predicted_iou = box_iou(full_truth_box, predicted_box)
        tconf[batch, truth_n, :, truth_h, truth_w] = predicted_iou
        conf_learning_scale[batch, truth_n, :, truth_h, truth_w] = 10.0

NameError: name 'batch_size' is not defined

In [37]:
r = np.random.rand(3,2)

In [39]:
r.shape

(3, 2)

In [40]:
rr = np.max(r, axis=0)

In [41]:
rr.shape

(2,)